In [ ]:
!pip install yfinance
!pip install bs4
!pip install nbformat
!pip install --upgrade plotly

In [173]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [174]:
import plotly.io as pio
pio.renderers.default = "iframe"

In [175]:
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [176]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021-06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

In [177]:
ticker = "TSLA"
stock = yf.Ticker(ticker)


In [178]:
hist = stock.history(period='max')
tesla_data = pd.DataFrame(hist)

In [179]:
tesla_data.reset_index(inplace=True)
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


In [180]:
html_data = requests.get('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm')

In [181]:
html_data = html_data.text
soup = BeautifulSoup(html_data,'html.parser')

In [188]:
tesla_revenue = pd.DataFrame(columns=['Date','Revenue'])
tale = soup.find_all('table')[1]
header = [header.text for header in tale.find_all('th')]
data = []
for i in tale.find('tbody').find_all('tr'):
    column = i.find_all('td')
    date = column[0].text.strip()
    revenue = column[1].text.strip()
    data.append({"Date": date, "Revenue": revenue})
tesla_revenue = pd.DataFrame(data)
tesla_revenue['Revenue'].replace('', pd.NA, inplace=True)
tesla_revenue.dropna(subset=['Revenue'], inplace=True)

In [191]:
tesla_revenue['Revenue'] = tesla_revenue['Revenue'].replace('[\$,]', '', regex=True).astype(float)

In [193]:
tesla_revenue.tail()

,Date,Revenue
48,2010-09-30,31.0
49,2010-06-30,28.0
50,2010-03-31,21.0
52,2009-09-30,46.0
53,2009-06-30,27.0


In [194]:
ticker2 = 'GME'
stock2 = yf.Ticker(ticker2)

In [195]:
html_data2 = requests.get('https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0220ENSkillsNetwork23455606-2021-01-01')
html_data2 = html_data2.text

In [196]:
gme_data = pd.DataFrame()
stock2_history = stock2.history('max')
gme_data = pd.concat([gme_data, stock2_history], ignore_index=True)
gme_data.reset_index(inplace=True)
if 'Date' not in gme_data.columns:
    gme_data.rename(columns={'index': 'Date'}, inplace=True)
gme_data['Date'] = pd.to_datetime(gme_data['Date'])
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,1970-01-01 00:00:00.000000000,1.620129,1.693350,1.603296,1.691667,76216000,0.0,0.0
1,1970-01-01 00:00:00.000000001,1.712707,1.716074,1.670626,1.683250,11021600,0.0,0.0
2,1970-01-01 00:00:00.000000002,1.683250,1.687458,1.658001,1.674834,8389600,0.0,0.0
3,1970-01-01 00:00:00.000000003,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,1970-01-01 00:00:00.000000004,1.615920,1.662210,1.603296,1.662210,6892800,0.0,0.0


In [197]:
soup2 = BeautifulSoup(html_data2,'html.parser')

In [209]:
gme_revenue = pd.DataFrame(columns=['Date','Revenue'])
tables = soup.find_all('table')
for table in tables:
    headers = [header.text.strip().lower() for header in table.find_all('th')]
    if 'date' in headers and 'revenue' in headers:
        revenue_table = table
        break
header = [header.text for header in table.find_all('th')]
data = []
for i in table.find('tbody').find_all('tr'):
    column = i.find_all('td')
    date = column[0].text.strip()
    revenue = column[1].text.strip()
    data.append({"Date": date, "Revenue": revenue})

gme_revenue = pd.DataFrame(data)
gme_revenue['Revenue'].replace('', pd.NA, inplace=True)
gme_revenue.dropna(subset=['Revenue'], inplace=True)
gme_revenue['Revenue'] = gme_revenue['Revenue'].replace('[\$,]', '', regex=True).astype(float)


In [210]:
gme_revenue.tail()

,Date,Revenue


In [ ]:
make_graph(tesla_data,tesla_revenue,'Tesla')

In [164]:
make_graph(gme_data, gme_revenue, 'GameStop')